## Universidad Autonoma de Nuevo León
## Facultad de Ciencias Físico Matemáticas
### Maestría en Ciencia de Datos
### Aprendizaje Automático
#### Eduardo David González Flores
##### 1515489

## Tarea (8 puntos)
- Aplicar al menos un algoritmo no supervisado a tus datos para encontrar estructuras subyacentes
- Elegir alguna métrica para determinar número de grupos, usarla y discutirla
- Busca alguna revista científica que publique trabajos relacionados con el tuyo
- Crea artículo mediante Latex con base en los lineamientos de la revista elegida y redacta ahí tus resultados, discusiones y bibliografía
- Sube el código de tu tarea, los archivos de Latex y el PDF del artículo en tu repositorio, claramente diferenciados